# SelfImplementedNeuralNetwork
A neural network implemented using only numpy.

In [148]:
import numpy as np

## Initiliase
Initialises the weights and biases of the network as all zero ~~with uniform distributions from -1 to 1~~. The network comprises 4 layers having 28² = 784, 128, 128 and 10 neurons respectively.

In [149]:
def init_network():
    # The layer sizes
    n0 = 28*28
    n1 = 128
    n2 = 128
    n3 = 10
    
    init_weights = lambda m, n: np.random.uniform(-1,1,(n,m))
    init_biases = lambda m: np.random.uniform(-1,1,(m,))
    # init_weights = lambda m, n: np.zeros((n,m)) # <- Attention: This function flips n and m for convenient input
    #  init_biases = lambda m: np.zeros((m,))
    
    layer0_1_weights = init_weights(n0,n1)
    layer0_1_biases = init_biases(n1)
    layer0_1 = [layer0_1_weights, layer0_1_biases]
    
    layer1_2_weights = init_weights(n1,n2)
    layer1_2_biases = init_biases(n2)
    layer1_2 = [layer1_2_weights, layer1_2_biases]
    
    layer2_3_weights = init_weights(n2,n3)
    layer2_3_biases = init_biases(n3)
    layer2_3 = [layer2_3_weights, layer2_3_biases]
    
    return [layer0_1, layer1_2, layer2_3]

## Activation functions
The *ReLU* (Rectified Linear Unit) activation function is defined as $\text{ReLU}(x)=\text{max}(0,x)$ and is the go-to activation function for hidden layers.

In [150]:
def relu(x):
    return np.maximum(0,x)

The *Sotfmax* acitvation function is a generalization of the *Sigmoid* function. It is applied to a 1D-Array of values which are squished to the interval $[0,1)$ such that the sum of all values is 1. Thereby, we get a true probability distribution which is quite convenient. It is defined as

$$\sigma(x)_i=\frac{e^{x_i}}{\sum_{j}e^{x_j}}\quad\forall\;i.$$

To be compatible with forward propagation on a 2D-Array of multiple training examples we extend the function to also take 2D-Arrays as a parameter and perform the softmax row-wise. (How this works in detail is annotated in the comments.)

Moreover, if the arrays contain big values we quickly get an overflow for `np.exp`. Therefore, we subtract the maximum value of each row (or for the 1D case: simply the maximum) from all values which doesn't alter the endresult of our computation, but rids us of some possible (pseudo-)infinities along the way. 

In [151]:
def softmax(x):
    # If x is a 2D-array of multiple training exmaples, we transpose so that axis=0 is the axis of a single 
    # training example. (We can't simply use axis=1 because that would break if we use the function for a 
    # 1D-array, i.e. only one training example. Transposing first and then using axis=0 works in both cases.)
    x = x.T
    # To avoid overflow of np.exp (doesn't alter the value)
    x = x - np.max(x, axis=0)
    # The main calculation
    result = np.exp(x) / np.sum(np.exp(x), axis=0)
    # Tranpsose back to normal form where each row is a training example (only makes a difference if array is 2D;
    # doesn't change anything if array is 1D.)
    return result.T

## Forward propagation
Here we implement forward propagation. This function can either take a 1D-Array of a single training example or a 2D-Array of multiple training examples where each row is a training example. The transformations (`np.array.T`) are necessary so that the dot product and the vector addition work in the 2D case. They can be ignored in the 1D case.

For backpropagation we not only need the endresult, but all intermediate results. If the option `intermediate_results` is set to `True` (default is `False`) we return a tuple $(y^{(1)},o^{(1)},y^{(2)},o^{(2)},y^{(3)},o^{(3)})$. Otherwise we simply return $o^{(3)}$.

In [310]:
def feed_forward(data, network, intermediate_results=False):
    x = data

#   # How it would work iteratively:
#   layer1 = []
#   for weights, bias in zip(network[0][0], network[0][1]):
#      neuron = relu(np.dot(weights, layer0) + bias)
#      layer1.append(neuron)  
#   layer1 = np.array(layer1)
    
    if len(x.shape) == 1:
        y1 = np.dot(network[0][0], x.T).T + network[0][1]        
        
        o1 = relu(y1)

        y2 = np.dot(network[1][0], o1.T).T + network[1][1]
        o2 = relu(y2)

        y3 = np.dot(network[2][0], o2.T).T + network[2][1]
        o3 = softmax(y3)
    elif len(x.shape) == 2:
        y1 = np.einsum('ji,bi->bj', network[0][0], x) + network[0][1]
        o1 = relu(y1)

        y2 = np.einsum('ji,bi->bj', network[1][0], o1) + network[1][1]
        o2 = relu(y2)

        y3 = np.einsum('ji,bi->bj', network[2][0], o2) + network[2][1]
        o3 = softmax(y3)
    
    if not intermediate_results:
        return o3
    else:
        return y1, o1, y2, o2, y3, o3

## Get data
We use the mnist handwritten digit classification data which is shipped with keras.

In [153]:
network = init_network()

In [154]:
import tensorflow as tf
(x_train_raw, y_train), (x_test_raw, y_test) = tf.keras.datasets.mnist.load_data()

Flatten the data to make it a well-behaved input for our neural network:

In [155]:
x_train = x_train_raw.reshape(60000, 28*28)
x_test = x_test_raw.reshape(10000, 28*28)

The feed forward function can either take a single example or an array of examples:

In [156]:
feed_forward(x_train[10], network) == feed_forward(x_train, network)[10]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

## Cost function
First, the loss function, i.e. the error for a single training example. There are a multitude of loss functions, though there is an "industry-standard" for each type of problem. While the simpler and better-known MSE (*mean squared error*) is used for regression problems, one uses *cross entropy loss* for categorization problems (which fits our case precisely).

Given a prediction $\hat{y_i}$ and the correct solution $y_i$ (in our case for $0\leq i<10$), we can define *cross entropy loss* as 
$$H(y,\hat{y})=-\sum_{i=0}^9 y_i\cdot \text{log}(\hat{y}_i).$$
Since we know that $y_i$ is zero for all but one $i$ (for which it has value 1), we can simplify this to
$$H(y,\hat{y})=-\text{log}(\hat{y}_k)$$
where $k$ is the index of the correct solution.

In [157]:
def cross_entropy_loss(y_hat, y):
    y_val = y_hat[y]
    # To avoid division by zero
    if y_val == 0:
        # #The numpy epsilon (np.finfo(float).eps) is apparently not really 
        # # the smallest number possible; by trying out I found the limit for 
        # # np.log to not throw an error was at about eps*10^(-307).
        # y_val = np.finfo("float64").eps*10**-307
        # Update: 
        # np.spacing(0) gives the smallest number bigger than 0
        y_val = np.spacing(0)
    return -np.log(y_val)

In [158]:
result = feed_forward(x_train, network)

In [159]:
cross_entropy_loss(result[10], y_train[10])

744.4400719213812

Now, we define the cost function, i.e. the error for a whole training set (or subset). In other words
$$E(Y,\hat{Y})=\sum_{\forall\;y,\,\hat{y}}H(y,\hat{y}).$$

In [160]:
def cross_entropy_cost(y_hats, ys):
    return np.sum(cross_entropy_loss(y_hat, y) for y_hat, y in zip(y_hats, ys))

In [161]:
cross_entropy_cost(result, y_train)

40502680.76254245

## Backpropagation
Now, finally the training will be implemented.

### Definitions

- Let the layers be labeled as $n=0,\dots,3$.
- Let $x_i$ be the $i$-th $x$-value, that is the $i$-th input of the network ($0\leq i<28^2$).
- Let $y_i^{(n)}$ be the values of the $i$-th neuron of layer $n$ *before* the activation function ($0\leq i<128$ for $n\in\{1,2\}$, $0\leq i<10$ for $n=3$).
- Let $o_i^{(n)}$ be the values of the $i$-th neuron of layer $n$ *after* the activation function.
    - $o_i^{(n)}=\text{ReLU}(y_i^{n})$ for $n\in\{1,2\}$
    - $o_i^{(3)}=\sigma(y^{(3)})_i$
- Let $w_{ij}^{(n)}$ represent the weight on the connection of the $i$-th neuron in layer n to the $j$-th neuron in layer n+1. (Can be found in `network[n][0][j][i]`.)  
- Let $b_i^{(n)}$ represent the bias of the $i$-th neuron in layer n+1. (Can be found in `network[n][1][i]`.)
- Within the context of the cost/loss function we use the following definitions:
    - $x$ and $y$ are the input and expected output of a single training example;
    - $X$ and $Y$ are the set (or a subset) of training examples; 
    - $\hat{y}$ is the actual output of our network, in other words: $\hat{y}=o^{(3)}$; 
    - $\hat{Y}$ is the set of actual outputs for a set of inputs $X$.

### Stochastic Gradient Descent

We split the computation of the gradient into six parts:
- Gradient of the layer 2 to 3 weights ($w^{(2)}$)
- Gradient of the layer 3 biases ($b^{(2)}$)
- Gradient of the layer 1 to 2 weights ($w^{(1)}$)
- Gradient of the layer 2 biases ($b^{(1)}$)
- Gradient of the layer 0 to 1 weights ($w^{(0)}$)
- Gradient of the layer 1 biases ($b^{(0)}$)

#### Layer 2 to 3 weights

##### Overview
Our goal is to determine
$$\frac{\partial}{\partial w_{ij}^{(2)}} \;E(Y,\hat{Y})$$
$$=\quad\sum_{\forall y,\hat{y}} \frac{\partial}{\partial w_{ij}^{(2)}} \;H(y,\hat{y})$$
$$=\quad\sum_{\forall y,\hat{y}} \frac{\partial}{\partial w_{ij}^{(2)}} \;\left(-\text{log}(\hat{y}_k)\right),$$
where $k$ is the correct solution for a given sample.

##### Cost function derivative with respect to a single weight
Hence, let us consider
$$\frac{\partial}{\partial w_{ij}^{(2)}} \;\left(-\text{log}(\hat{y}_k)\right)$$
$$=\quad \frac{\partial}{\partial w_{ij}^{(2)}} \;\left(-\text{log}(o^{(3)}_k)\right)$$
$$=\quad \frac{\partial}{\partial o^{(3)}_k}\;\left(-\text{log}(o^{(3)}_k)\right)\cdot\frac{\partial o^{(3)}_k}{\partial w_{ij}^{(2)}}$$
$$=\quad -\frac{1}{o^{(3)}_k}\cdot \frac{\partial}{\partial w_{ij}^{(2)}} \;\sigma(y^{(3)})_k$$
$$=\quad -\frac{1}{o^{(3)}_k}\cdot \frac{\partial}{\partial w_{ij}^{(2)}} \;\frac{e^{y^{(3)}_k}}{\sum_{\forall r}e^{y^{(3)}_r}}.$$
$$=\quad -\frac{1}{o^{(3)}_k}\cdot \left(\sum_{\forall s}\frac{\partial}{\partial y^{(3)}_s} \;\frac{e^{y^{(3)}_k}}{\sum_{\forall r}e^{y^{(3)}_r}} \cdot \frac{\partial y^{(3)}_s}{\partial w_{ij}^{(2)}} \right).$$

To simplify it more, let us focus on
$$\frac{\partial}{\partial y^{(3)}_s} \;\frac{e^{y^{(3)}_k}}{\sum_{\forall r}e^{y^{(3)}_r}}$$
$$=\quad \frac{\partial}{\partial e^{y^{(3)}_s}} \;\frac{e^{y^{(3)}_k}}{\sum_{\forall r}e^{y^{(3)}_r}} \cdot \frac{\partial e^{y^{(3)}_s}}{\partial y^{(3)}_s}$$
$$=\quad e^{y^{(3)}_s}\cdot\frac{\partial}{\partial e^{y^{(3)}_s}} \;\frac{e^{y^{(3)}_k}}{\sum_{\forall r}e^{y^{(3)}_r}}$$
$$=\quad e^{y^{(3)}_s} \cdot 
\frac{ 
    \frac{\partial}{ \partial e^{y^{(3)}_s} } \; e^{y^{(3)}_k} \cdot \left(\sum_{\forall r}e^{y^{(3)}_r}\right) 
    -  e^{y^{(3)}_k} \cdot \frac{\partial}{ \partial e^{y^{(3)}_s} } \; \left(\sum_{\forall r}e^{y^{(3)}_r}\right)
}{
    \left(\sum_{\forall r}e^{y^{(3)}_r}\right)^2
}.$$

Here, we need to make a distinction between $s=k$ and $s\neq k$. If $s=k$, we get
$$e^{y^{(3)}_s} \cdot
\frac{ 
    \sum_{\forall r}e^{y^{(3)}_r} 
    -  e^{y^{(3)}_k}
}{
    \left(\sum_{\forall r}e^{y^{(3)}_r}\right)^2
}$$
$$=\quad 
\frac{
    e^{y^{(3)}_s}
}{
    \sum_{\forall r}e^{y^{(3)}_r}
}
\cdot
\frac{
    \sum_{\forall r}e^{y^{(3)}_r} 
    -  e^{y^{(3)}_k}
}{
    \sum_{\forall r}e^{y^{(3)}_r}
}
$$
$$=\quad \sigma(y^{(3)})_s \cdot \left( 1 - \sigma(y^{(3)})_k \right)$$
$$=\quad o^{(3)}_s \cdot \left( 1 - o^{(3)}_k \right).$$

If $s\neq k$, we get
$$e^{y^{(3)}_s} \cdot
\frac{ 
    -  e^{y^{(3)}_k}
}{
    \left(\sum_{\forall r}e^{y^{(3)}_r}\right)^2
}$$
$$=\quad - \sigma(y^{(3)})_s \cdot \sigma(y^{(3)})_k$$
$$=\quad - o^{(3)}_s \cdot o^{(3)}_k.$$

Furthermore, we need to evaluate
$$\frac{\partial y^{(3)}_s}{\partial w_{ij}^{(2)}}.$$
We know that
$$y^{(3)}_s = \sum_{\forall t} w_{ts}^{(2)}o^{(2)}_t + b_s^{(2)}.$$
If $j=s$, we have
$$\frac{\partial y^{(3)}_s}{\partial w_{ij}^{(2)}} = o^{(2)}_i.$$
Else, if $j\neq s$, we have
$$\frac{\partial y^{(3)}_s}{\partial w_{ij}^{(2)}} = 0.$$

The equation we considered before was
$$-\frac{1}{o^{(3)}_k}\cdot \left(\sum_{\forall s}\frac{\partial}{\partial y^{(3)}_s} \;\frac{e^{y^{(3)}_k}}{\sum_{\forall r}e^{y^{(3)}_r}} \cdot \frac{\partial y^{(3)}_s}{\partial w_{ij}^{(2)}} \right),$$
where -- as we see from our last result -- all terms but one turn out to be zero. What remains is
$$-\frac{1}{o^{(3)}_k}\cdot \left(\frac{\partial}{\partial y^{(3)}_j} \;\frac{e^{y^{(3)}_k}}{\sum_{\forall r}e^{y^{(3)}_r}}\cdot o_i^{(2)}\right).$$
Now, we have to distinguish between $j=k$ and $j\neq k$, which yields us for $j=k$
$$\frac{\partial}{\partial w_{ij}^{(2)}} \;H(y,\hat{y}) = 
    -\frac{1}{o^{(3)}_k}
    \cdot 
    o^{(3)}_j \cdot \left( 1 - o^{(3)}_k \right)
    \cdot 
    o_i^{(2)},$$
and for $j\neq k$
$$\frac{\partial}{\partial w_{ij}^{(2)}} \;H(y,\hat{y}) = 
    \frac{1}{o^{(3)}_k}
    \cdot
    o^{(3)}_j \cdot o^{(3)}_k
    \cdot 
    o_i^{(2)}$$
$$=\quad o^{(3)}_j\cdot o_i^{(2)}.$$

##### Unifying it into a single matrix
Our weights matrix $w^{(2)}$ has dimensions $n^{(3)} \times n^{(2)}$. (This means the indices $i$ and $j$ in $w_{ij}^{(2)}$ are actually the wrong way around.) Let us call the gradient of $H(y,\hat y)$ w.r.t. the weight matrix 
$$\nabla_{w^{(2)}} \,H(y,\hat y).$$ 
This gradient shall be a $n^{(3)} \times n^{(2)}$ matrix as well. (Hence, contrary to popular usage, we extend the notion of gradient from simply being a vector to also being a matrix; this is useful because we also treat the weights as matrices which simplifies both updating the weights according to the gradient and computing the gradient non-iteratively easier.)

There is a distinction to be made between $j=k$ and $j\neq k$, i.e. the $k$-th row in the gradient matrix needs a special treating. Luckily, considering the case $j=k$ as derived above we see
$$\frac{\partial}{\partial w_{ij}^{(2)}} \;H(y,\hat{y}) = 
    -\frac{1}{o^{(3)}_k}
    \cdot 
    o^{(3)}_j \cdot \left( 1 - o^{(3)}_k \right)
    \cdot 
    o_i^{(2)}$$
$$= \quad
    -\frac{1}{o^{(3)}_k} \cdot o^{(3)}_j \cdot o_i^{(2)}
    \; + \;
    \frac{1}{o^{(3)}_k} \cdot o^{(3)}_j \cdot o^{(3)}_k \cdot o_i^{(2)}$$
$$= \quad
    - o_i^{(2)}
    \; + \;
    o^{(3)}_j \cdot o_i^{(2)}$$
where the second summand coincides with the case $j\neq k$. 

Thus, we are able to split the unification of the many partial derivatives into a single gradient matrix into two steps
1. Compute the matrix where the element in the $i$-th column and $j$-th row is $o^{(3)}_j \cdot o_i^{(2)}$.
2. Subtract from the $k$-th row a (row) vector where the $i$-th column is $o_i^{(2)}$.

The computation in step 1 is
$$\quad o^{(3)} \otimes o^{(2)}$$
where $a\otimes b = a\cdot b^T$ denotes the *outer product*, *matrix product* or *tensor product* of two vectors (called `np.outer`). In the second step, we simply subtract $o^{(2)}$ (as a row vector) from the $k$-th column of the result of step 1. (Expressing this operation mathematically is not necessary, here we just have to consider implementing it efficiently.) 

Lastly, we can sum this matrix for multiple training examples
$$\nabla_{w^{(2)}} \;E(Y,\hat{Y}) = \sum_{\forall\;y,\hat y} \nabla_{w^{(2)}} \;H(y,\hat y).$$

#### Layer 3 biases
Here, we examine
$$\frac{\partial}{\partial b_{j}^{(2)}} \;E(Y,\hat{Y})$$
$$=\quad\sum_{\forall y,\hat{y}} \frac{\partial}{\partial b_{j}^{(2)}} \;H(y,\hat{y}),$$
which can be derived in a very analogous way. The key difference arises when determining
$$\frac{\partial y^{(3)}_s}{\partial b_{j}^{(2)}}.$$
If $j=s$, we have
$$\frac{\partial y^{(3)}_s}{\partial b_{j}^{(2)}} = 1,$$
instead of $o_i^{(2)}$ which we would have got for for $w_{ij}^{(2)}$. Else, if $j\neq s$, we still have
$$\frac{\partial y^{(3)}_s}{\partial b_{j}^{(2)}} = 0.$$
Replacing every occasion of $o_i^{(2)}$ with $1$, we see that $\nabla_{b^{(2)}} \;H(y,\hat y)$ is a $n^{(3)}$-dimensional vector where the $j$-th row is $o_j^{(3)}$ if $j\neq k$ and $o_j^{(3)}-1$ if $j=k$.

In [246]:
def stochastic_gradient_descent(x_train, y_train, network, batch_size=50, epochs=3):
    
    # Loop through epochs
    for i in range(epochs):
        
        #####################################
        ### Case of a single training example
        x = x_train[0]
        k = y_train[0]
        
        y1, o1, y2, o2, y3, o3 = feed_forward(x, network, intermediate_results=True)
        
        ## Layer 2to3 weights
        # Step 1
        w2_grad = np.outer(o3, o2)
        w2_grad_prev = w2_grad
        # Step 2
        w2_grad[k] -= o2
                    
        ## Layer 3 biases
        # Step 1
        b2_grad = o3
        # Step 2
        b2_grad[k] -= 1
        
        ## Update
        #network[2][0] -= w2_grad
        #network[2][1] -= b2_grad
        #####################################
        
        ## Multiple training examples at once
        y1, o1, y2, o2, y3, o3 = feed_forward(x_train[:100], network, intermediate_results=True)
        
        ## Layer 2to3 weights
        # Step 1
        w2_grads = np.einsum('rj,ri->rji',o3,o2)
        w2_grads_prev = w2_grads
        # Step 2
        for i in range(100):
            w2_grads[i][y_train[i]] -= o2[i]
        
        return w2_grad_prev, w2_grads_prev
    
    # Layer 1to2 weights
    
    # Layer 2 biases
    
    # Layer 0to1 weights
    
    # Layer 1 biases

In [293]:
network = init_network()

In [308]:
y1, o1, y2, o2, y3, o3 = feed_forward(x_train[:100], network, intermediate_results=True)

In [302]:
w2_grad = np.einsum('rj,ri->rji',o3,o2)

In [307]:
y1_s, o1_s, y2_s, o2_s, y3_s, o3_s = feed_forward(x_train[0], network, intermediate_results=True)

True


In [304]:
print("y1", np.array_equal(y1[0], y1_s))
print("o1", np.array_equal(o1[0], o1_s))
print("y2", np.array_equal(y2[0], y2_s))
print("o2", np.array_equal(o2[0], o2_s))
print("y3", np.array_equal(y3[0], y3_s))
print("o3", np.array_equal(o3[0], o3_s))

y1 False
o1 False
y2 False
o2 False
y3 False
o3 True


(128,)

In [250]:
a,b = stochastic_gradient_descent(x_train, y_train, network)

744.4400719213812


In [251]:
a.shape

(10, 128)

In [252]:
np.sum(a == b[0])

1164

In [196]:
o3 = feed_forward(x_train[0], network)

In [197]:
cross_entropy_loss(o3,5)

744.4400719213812